In [1]:
import numpy as np
import astropy.constants as const
import astropy.units as units
import bokeh.plotting as blt
import bokeh
bokeh.io.output_notebook()

Loading BokehJS ...

# q3

In [2]:
#units are solar masses, kpc and Gyr
G=const.G.to((units.kpc**3)/(const.M_sun*(units.Gyr**2))).value
def vCirc(rad,Ma,a):
    return np.sqrt(G*rMass(rad,Ma,a)/rad)
def rMass(rad,Ma,a): #mass within radius r
    return Ma*( np.log(1+(rad/a)) - (rad/(a+rad)) )
def phi(rad,Ma,a):
    return -G*Ma*np.log(1+(rad/a))/rad
def effPhi(rad,Ma,a,l):
    return phi(rad,Ma,a) + ((l**2)/(2*(rad**2)))
def energy(rad,rDot,Ma,a,l):
    return (rDot**2)/2 + effPhi(rad,Ma,a,l)
def eCirc(rad,Ma,a,l): #energy assuming a circular orbit
    return (vCirc(rad,Ma,a)**2)/2 + phi(rad,Ma,a)
def lMax(rad,Ma,a,E):
    return rad*np.sqrt(2*(E-phi(rad,Ma,a)))

# 3.a

In [3]:
Ma=1e11
a=2
testR=np.linspace(0.1,25,100)
plotW=400
plotH=200
vCircPlot=blt.figure(width=plotW,height=plotH,
                     x_axis_label='r (kpc)',y_axis_label='v_c (kpc/Gyr)',
                     y_axis_type="log")
vCircPlot.line(testR,vCirc(testR,Ma,a),line_width=2)
mPlot=blt.figure(width=plotW,height=plotH,
                 x_axis_label='r (kpc)',y_axis_label='M_r (Msun)',
                 y_axis_type="log")
mPlot.line(testR,rMass(testR,Ma,a),line_width=2)
grid=bokeh.layouts.gridplot([[vCircPlot,mPlot]])
blt.show(grid)

# 3.b

In [4]:
l=1e3
effPhiPlot=blt.figure(width=plotW,height=2*plotH,
                 x_axis_label='r (kpc)',y_axis_label='energy',
                 y_range=(-1e5,1e5),x_range=(0,25))
effPhiPlot.line(testR,effPhi(testR,Ma,a,l),line_width=2,legend='effective potential')
effPhiPlot.line(testR,eCirc(testR,Ma,a,l),line_width=2,color='green',legend='energy of circular orbit')
#effPhiPlot.ray(x=[0], y=[0], length=0, angle=0, line_width=2)
#effPhiPlot.ray(x=[0], y=[energy(np.zeros_like(testR),testR,Ma,a,l)]
#               ,length=0, angle=np.pi/2, line_width=2)
blt.show(effPhiPlot)

# 3.c

In [106]:
E=5 # for -ve E there exists a maximum l possible
lMaxPlot=blt.figure(width=plotW,height=2*plotH,
                 x_axis_label='r (kpc)',y_axis_label='l_max',
                 x_range=(0,25))
lMaxPlot.line(testR,lMax(testR,Ma,a,E),line_width=2)
blt.show(lMaxPlot)

# q4

In [55]:
def timeIntegrand(rad,Ma,a,l,E):
    return 2/(2*(E-phi(rad,Ma,a)) + l**2/rad**2)**0.5
def precessIntegrand(rad,Ma,a,l,E):
    return 2*l/(rad**2 * (2*(E-phi(rad,Ma,a)) + l**2/rad**2)**0.5)
def rootBisection(lowBound,highBound,function,Ma,a,l,E):
    low=lowBound
    mid=0.5*(lowBound+highBound)
    high=highBound
    diff=1e10 #just a large number to get started
    count=0
    while((count<100) & (np.abs(diff/E)>0.001)):
        #print('count: ',count,' diff/E: ',diff/E)
        #print('low: ',low,' high: ',high)
        count+=1
        lowDiff=function(low,Ma,a,l)-E
        highDiff=function(high,Ma,a,l)-E
        diff=function(mid,Ma,a,l)-E
        if (np.sign(lowDiff*diff) == 1): #low and mid point on same side of root
            low=mid
            mid=0.5*(low+high)
        elif (np.sign(highDiff*diff) == 1): # high and mid point on same side of root
            high=mid
            mid=0.5*(low+high)
        #else:
            #print('something went wrong, 0 or 2+ roots in range')
    if (count==100):
        #print('did not converge')
        return -1
    return mid
def trapIntegrate(lowBound,highBound,nStep,function,Ma,a,l,E):
    integral=0
    step=np.linspace(lowBound,highBound,nStep+1)
    dStep=step[1]-step[0]
    for i in range(nStep):
        integral+=dStep*(function(step[i],Ma,a,l,E)+function(step[i+1],Ma,a,l,E))/2
    return integral
def plotImage(imageData,cbarName,cbarLow,cbarHigh):
    cmap = bokeh.models.LinearColorMapper(palette="PuOr11", low=cbarLow, high=cbarHigh)
    cbar = bokeh.models.ColorBar(color_mapper=cmap, border_line_color=None, location=(0,0),title=cbarName)
    plot=blt.figure(width=plotW,height=plotH,
                     y_axis_label='l',x_axis_label='E',
                     y_range=(ls[0],ls[-1]),x_range=(Es[0],Es[-1]))
    plot.add_layout(cbar, 'right')
    plot.image(image=[imageData],y=ls[0],x=Es[0],dh=ls[-1]-ls[0],dw=Es[-1]-Es[0],color_mapper=cmap)
    return plot
nL=128
nE=128
ls=np.linspace(5e2,5e3,nL)
Es=np.linspace(-1e5,1e5,nE)

# 4.a

In [74]:
def findTurnRad(l,E):
    minRad=-1
    maxRad=-1
    if(E>0): #only one root
        minRad=rootBisection(1,1e5,effPhi,Ma,a,l,E)
    elif(E<0):
        circRad=rootBisection(1,1e5,eCirc,Ma,a,l,E) #if 2 roots need to find center point to bisect around
        if (eCirc(circRad,Ma,a,l) < effPhi(circRad,Ma,a,l)): #no solutions
            return -1,-1
        #print('r_circ: ',circRad)
        minRad=rootBisection(1,circRad,effPhi,Ma,a,l,E)
        maxRad=rootBisection(circRad,1e5,effPhi,Ma,a,l,E)
        #print('r_apo: ',minRad,' and r_peri: ',maxRad)
    return minRad,maxRad

apos=np.zeros((nL,nE))
peris=np.zeros((nL,nE))
for i in range(nL):
    for j in range(nE):
        minRad,maxRad=findTurnRad(ls[i],Es[j])
        apos[i,j]=minRad
        peris[i,j]=maxRad

#print('apos: ',apos)
#print('peris: ',peris)

apoPlot=plotImage(apos,'r_apo',0,20)
periPlot=plotImage(peris,'r_peri',0,100)

grid=bokeh.layouts.gridplot([[apoPlot,periPlot]])
blt.show(grid)

# 4.b

In [108]:
#l=1e3
#E=-5e4
#minRad,maxRad=findTurnRad(l,E)
#Tr=trapIntegrate(minRad,maxRad,50,timeIntegrand,Ma,a,l,E)
#print(Tr)
Trs=np.zeros((nL,nE))
dPsi=np.zeros((nL,nE))
for i in range(nL):
    for j in range(nE):
        minRad,maxRad=findTurnRad(ls[i],Es[j])
        if (maxRad==-1): #no circular orbit
            Trs[i,j]=0
            dPsi[i,j]=0
            continue
        Trs[i,j]=trapIntegrate(minRad,maxRad,50,timeIntegrand,Ma,a,ls[i],Es[j])
        dPsi[i,j]=trapIntegrate(minRad,maxRad,50,precessIntegrand,Ma,a,ls[i],Es[j]) % 2*np.pi

tPlot=plotImage(Trs,'period',0,1)
psiPlot=plotImage(dPsi,'precession',0,2*np.pi)

grid=bokeh.layouts.gridplot([[tPlot,psiPlot]])
blt.show(grid)

/Users/Zephyr/Astro/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app
/Users/Zephyr/Astro/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in double_scalars


# q5

In [102]:
def dPhi_dR(rad,Ma,a):
    return G*rMass(rad,Ma,a)/rad**2
def dPhiEff_dR(rad,Ma,a,l):
    #print('dphi_dr: ',dPhi_dR(rad,Ma,a))
    #print('l2/r3: ',l**2/rad**3)
    return dPhi_dR(rad,Ma,a)-l**2/rad**3
class particleClass():
    def __init__(self):
        self.r=0
        self.rDot=0
        self.psiDot=0
    def leapFrog(self,dt):
        self.r+=self.rDot*dt
        #print('dt: ',dt)
        #print('r: ',self.r)
        rDotDot=-dPhiEff_dR(self.r,Ma,a,self.r**2 * self.psiDot)
        psiDotDot=-2*self.rDot*self.psiDot/self.r
        #print('rDotDot: ',rDotDot)
        #print('psiDotDot: ',psiDotDot)
        self.rDot+=rDotDot*dt
        self.psiDot+=psiDotDot*dt
        
endTime=1
nSteps=10000
dt=endTime/nSteps
p=particleClass()
E=-8.5e4
l=1e3
rApo,rPeri=findTurnRad(l,E)
print('expected apo and peri: ',rApo,' and ',rPeri)
p.r=rPeri
p.psiDot=l/p.r**2
rHist=np.zeros(nSteps)
lHist=np.zeros(nSteps)
eHist=np.zeros(nSteps)
for step in range(nSteps):
    p.leapFrog(dt)
    rHist[step]=p.r
    lHist[step]=(p.r**2) * p.psiDot
    eHist[step]=0.5*((p.rDot**2) *((p.r*p.psiDot)**2)) + phi(p.r,Ma,a)
    
rPlot=blt.figure(width=plotW,height=plotH,
                     y_axis_label='r',x_axis_label='t',
                     y_range=(0.9*rApo,1.1*rPeri),x_range=(0,endTime))
rPlot.line(dt*np.arange(0,nSteps),rHist,line_width=2)
rPlot.line([0,endTime],[rApo,rApo])
rPlot.line([0,endTime],[rPeri,rPeri])

consPlot=blt.figure(width=plotW,height=plotH,
                     y_axis_label='E and l drift',x_axis_label='t',
                     y_range=(0,2),x_range=(0,endTime))
consPlot.line(dt*np.arange(0,nSteps),lHist/l,line_width=2)
consPlot.line(dt*np.arange(0,nSteps),eHist/E,line_width=2,color='green')
consPlot.line([0,endTime],[1,1])

grid=bokeh.layouts.gridplot([[rPlot,consPlot]])
blt.show(grid)

print(eHist)


expected apo and peri:  3.13606009632349  and  7.580192938863547


[   -91753.02930235    -88031.56327158    -81828.87652798 ...,
  39330517.4962898   39755309.58476266  40183725.83150367]
